In [2]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

from DataProcessing import RecsData as recs

In [4]:
offers = recs("../data/offering.txt")
json_offers = offers.to_json()
df_offers = pd.json_normalize(json_offers, meta=['address'])
df_offers.head()

4333it [00:00, 162026.99it/s]
100%|███████████████████████████████████████████████████████████████████████████| 4333/4333 [00:00<00:00, 30565.57it/s]

Initialize the dataset


,hotel_class,region_id,url,phone,details,type,id,name,address.region,address.street-address,address.postal-code,address.locality
0,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,113317,Casablanca Hotel Times Square,NY,147 West 43rd Street,10036,New York City
1,5.0,32655,http://www.tripadvisor.com/Hotel_Review-g32655...,,None,hotel,76049,Four Seasons Hotel Los Angeles at Beverly Hills,CA,300 S Doheny Dr,90048,Los Angeles
2,3.5,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,99352,Hilton Garden Inn Times Square,NY,790 Eighth Avenue,10019,New York City
3,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,93589,The Michelangelo Hotel,NY,152 West 51st Street,10019,New York City
4,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,,None,hotel,217616,The Muse Hotel New York,NY,130 West 46th Street,10036,New York City


In [5]:
reviews = recs("../data/review.txt")

149it [00:00, 1289.63it/s]

Initialize the dataset


878561it [00:37, 23247.49it/s]


In [16]:
reviews_json = reviews.to_json(prc=0.22)

100%|█████████████████████████████████████████████████████████████████████████| 193283/193283 [05:42<00:00, 564.88it/s]


In [15]:
pd.DataFrame.from_dict(reviews_json).head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,"December 17, 2012",147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,"December 17, 2012",147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,"December 18, 2012",147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,"December 17, 2012",147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,"December 17, 2012",147612823,False


In [13]:
new_reviews = pd.json_normalize(reviews_json, meta=['ratings', 'author'])
new_reviews.head()

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,ratings.service,ratings.cleanliness,...,ratings.rooms,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.num_type_reviews,author.id,author.location,ratings.check_in_front_desk,ratings.business_service_(e_g_internet_access)
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,0,"December 17, 2012",147643103,False,5.0,5.0,...,5.0,Papa_Panda,22.0,12.0,29.0,24.0,8C0B42FF3C0FA366A21CFD785302A032,Gold Coast,NaN,NaN
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,0,"December 17, 2012",147639004,False,5.0,5.0,...,5.0,Maureen V,2.0,NaN,2.0,NaN,E3C85CA9DBBBC77E0DB534ABE93E4713,"Sydney, New South Wales, Australia",NaN,NaN
2,“Great Stay”,This is a great property in Midtown. We two di...,December 2012,1762573,0,"December 18, 2012",147697954,False,4.0,5.0,...,4.0,vuguru,12.0,17.0,14.0,14.0,FB1032DECE1162CB3556D05F278AAFFD,Houston,NaN,NaN
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,August 2012,1762573,0,"December 17, 2012",147625723,False,5.0,5.0,...,5.0,Hotel-Designer,5.0,26.0,5.0,5.0,EC3E275EE7590694889C8C7EE0D13961,"Laguna Beach, CA",NaN,NaN
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,December 2012,1762573,0,"December 17, 2012",147612823,False,4.0,5.0,...,5.0,JamesE339,34.0,65.0,104.0,49.0,BA524A238B1171206691A6CC3F28F266,"Saint Louis, Missouri",NaN,NaN


In [51]:
new_reviews.filter(like='author')

,author.username,author.num_cities,author.num_helpful_votes,author.num_reviews,author.num_type_reviews,author.id,author.location
0,Papa_Panda,22.0,12.0,29.0,24.0,8C0B42FF3C0FA366A21CFD785302A032,Gold Coast
1,Maureen V,2.0,NaN,2.0,NaN,E3C85CA9DBBBC77E0DB534ABE93E4713,"Sydney, New South Wales, Australia"
2,vuguru,12.0,17.0,14.0,14.0,FB1032DECE1162CB3556D05F278AAFFD,Houston
3,Hotel-Designer,5.0,26.0,5.0,5.0,EC3E275EE7590694889C8C7EE0D13961,"Laguna Beach, CA"
4,JamesE339,34.0,65.0,104.0,49.0,BA524A238B1171206691A6CC3F28F266,"Saint Louis, Missouri"
...,...,...,...,...,...,...,...
131779,francoisregis,NaN,NaN,1.0,NaN,BBC5B72182DD5D2B3F4623778BFC0CA9,"Ault, France"
131780,,NaN,NaN,NaN,NaN,,
131781,クッキーチョコ,35.0,42.0,84.0,57.0,C8CB5DA995133091484E26C0AA0669D7,福岡
131782,PriorityTokyo,5.0,2.0,10.0,5.0,807C6AC10F63ABBF290BEF9A956C8396,Tokyo


In [14]:
data['sourceURLs'].head(10)

0    https://www.tripadvisor.com/Hotel_Review-g3243...
1    http://www.tripadvisor.com/Hotel_Review-g32172...
2    http://www.tripadvisor.com/Hotel_Review-g32172...
3    http://www.tripadvisor.com/Hotel_Review-g32172...
4    http://www.tripadvisor.com/Hotel_Review-g32172...
5    https://www.tripadvisor.com/Hotel_Review-g3958...
6    https://www.tripadvisor.com/Hotel_Review-g3958...
7    https://www.tripadvisor.com/Hotel_Review-g3981...
8    https://www.yellowbook.com/profile/microtel-in...
9    http://www.citysearch.com/profile/4378740/hopk...
Name: sourceURLs, dtype: object